# Imports

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline

In [3]:
df = pd.read_csv('../data/clean_final.csv')

In [4]:
df.head()

,index,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags,overlap_tags,target_class,body_clean
0,1,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,"['c++', 'qt', 'qt5', 'qtstylesheets', 'qpushbu...",775,2,2,{'c++'},c++,4,i have so many buttons on a dialog and i want ...
1,4,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,"['python', 'python-3.x', 'list', 'class', 'pri...",40,2,0,"{'python-3.x', 'python'}",python-3.x python,5,i am designing a deck class that has init meth...
2,8,53992234,2019-01-01 00:05:48,How to rearrange subplots so that one is under...,<p>I am trying to code two plots such that one...,"['python', 'matplotlib', 'subplot']",519,1,1,{'python'},python,5,i am trying to code two plots such that one pl...
3,10,53992248,2019-01-01 00:09:24,Function always returns 1,<p>I´m trying to write a simple branch predict...,"['c++', 'function']",150,1,21,{'c++'},c++,4,i m trying to write a simple branch predictor ...
4,11,53992252,2019-01-01 00:11:20,possible to speed up this query?,<p>I have the following query which takes a li...,"['sql', 'postgresql']",40,1,0,{'sql'},sql,0,i have the following query which takes a littl...


In [16]:
# Drop random and unnecesary'index' labeled column 
df = df.drop(['index'], axis = 1)

In [17]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags,overlap_tags,target_class,body_clean
0,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,"['c++', 'qt', 'qt5', 'qtstylesheets', 'qpushbu...",775,2,2,{'c++'},c++,4,i have so many buttons on a dialog and i want ...
1,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,"['python', 'python-3.x', 'list', 'class', 'pri...",40,2,0,"{'python-3.x', 'python'}",python-3.x python,5,i am designing a deck class that has init meth...
2,53992234,2019-01-01 00:05:48,How to rearrange subplots so that one is under...,<p>I am trying to code two plots such that one...,"['python', 'matplotlib', 'subplot']",519,1,1,{'python'},python,5,i am trying to code two plots such that one pl...
3,53992248,2019-01-01 00:09:24,Function always returns 1,<p>I´m trying to write a simple branch predict...,"['c++', 'function']",150,1,21,{'c++'},c++,4,i m trying to write a simple branch predictor ...
4,53992252,2019-01-01 00:11:20,possible to speed up this query?,<p>I have the following query which takes a li...,"['sql', 'postgresql']",40,1,0,{'sql'},sql,0,i have the following query which takes a littl...


In [23]:
df['CreationDate']= pd.to_datetime(df['CreationDate'])

In [25]:
# Check for nulls 
df.isnull().sum()

Id              0
CreationDate    0
Title           0
Body            0
Tags            0
ViewCount       0
AnswerCount     0
CommentCount    0
target_tags     0
overlap_tags    0
target_class    0
body_clean      1
dtype: int64

In [32]:
df[df['body_clean'].isnull()]

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags,overlap_tags,target_class,body_clean
91170,55234780,2019-03-19 06:21:07,1st data is officeIn and 2nd data is OfficeOut...,"<p><a href=""https://i.stack.imgur.com/P3SNS.pn...","['mysql', 'sql']",26,1,0,"{'sql', 'mysql'}",sql mysql,0,NaN


In [33]:
df.loc[91170, 'Body']

'<p><a href="https://i.stack.imgur.com/P3SNS.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/P3SNS.png" alt="enter image description here"></a></p>\n\n<pre><code>1476    5   2019-03-18 09:35:06.000\n1487    5   2019-03-18 13:19:53.000\n1488    5   2019-03-18 13:37:40.000\n1495    5   2019-03-18 15:09:38.000\n1497    5   2019-03-18 15:18:26.000\n1503    5   2019-03-18 17:34:46.000\n1504    5   2019-03-18 17:48:23.000\n1511    5   2019-03-18 19:14:51.000\n</code></pre>\n'

In [34]:
# drop row with any missing value
df = df.dropna()

In [35]:
df.isnull().sum()

Id              0
CreationDate    0
Title           0
Body            0
Tags            0
ViewCount       0
AnswerCount     0
CommentCount    0
target_tags     0
overlap_tags    0
target_class    0
body_clean      0
dtype: int64

# Exploratory Data Analysis (EDA)

- ngram bar graphs (most & least used words, bigrams)
    - Most used in python, sql etc 
- if word appears in more than 80%
    - vectorizer parameters
        - max_df()
- create % weekly graph showing frequency of questions for each language (use datetime and index )
- average length of words in question 


## One

In [38]:
# can't put creatingdate as index, which makes sense... 
df['CreationDate'].nunique()

460413

In [39]:
df.shape

(465054, 12)

In [51]:
df2 = df

In [69]:
(df2)

TypeError: 'DataFrame' object cannot be interpreted as an integer

In [65]:
def check_length(df, col):
    for i in len(df):
        len(df.loc[i,col])
    return df

In [63]:
check_length(df2, 'body_clean')

TypeError: 'DataFrame' object cannot be interpreted as an integer

In [49]:
df['body_length'] = df['body_clean'].map(lambda x: len(df.loc[x,'body_clean']))

KeyError: 'i have so many buttons on a dialog and i want to change style sheets of them under some conditions button object names are like below btn btn btn btn when i clicked one of these numerical buttons and later to another simple button i want to change first clicked numerical button style sheet how can i access that selected numerical button edit what i mean by picture i am trying to set colors of left column buttons has numerically ordered object names with right column buttons user will be clicked numerical buttons first and then color named buttons'

In [46]:
len(df.loc[1,'body_clean'])

1200

In [47]:
(df.loc[1,'body_clean'])

'i am designing a deck class that has init method with initially has an empty list then i append my cards to the list i am trying to create an x instance and access the shuffled version of the deck of cards i know there are many solutions posted already i just want to understand with my logic i am able to print the address of the card elements and not the deck itself while trying to debug i am not understanding whether print x cards in deck is printing the location or the x shuffle any good reference for pycharm debugging will also be highly appreciated suits hearts diamonds spades clubs ranks two three four five six seven eight nine ten jack queen king ace values two three four five six seven eight nine ten jack queen king ace class card def init self suit rank self suit suit self rank rank def str self return self rank of self suit class deck def init self self cards in deck for suit in suits for rank in ranks self cards in deck append card suit rank return self cards in deck def str

In [48]:
len('i am designing a deck class that has init method with initially has an empty list then i append my cards to the list i am trying to create an x instance and access the shuffled version of the deck of cards i know there are many solutions posted already i just want to understand with my logic i am able to print the address of the card elements and not the deck itself while trying to debug i am not understanding whether print x cards in deck is printing the location or the x shuffle any good reference for pycharm debugging will also be highly appreciated suits hearts diamonds spades clubs ranks two three four five six seven eight nine ten jack queen king ace values two three four five six seven eight nine ten jack queen king ace class card def init self suit rank self suit suit self rank rank def str self return self rank of self suit class deck def init self self cards in deck for suit in suits for rank in ranks self cards in deck append card suit rank return self cards in deck def str self for card in deck cards in deck return card return self cards in deck def shuffle cards self return random shuffle self cards in deck x deck print x cards in deck sep n print x shuffle cards sep n')

1200

In [43]:
X = df['body_clean']
y = df['target_class']

## Two 